IN Operator in Snowflake Snowpark

In [1]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from dotenv import load_dotenv
import os

load_dotenv()


connection_parameters = {
   "account": os.getenv('account_snow'),
   "user": os.getenv('user_snow'),
   "password":os.getenv('password_snow'),
   "role": "ACCOUNTADMIN",
   "database": "NEW_DB", 
   "schema": "PUBLIC"
  
}  

session = Session.builder.configs(connection_parameters).create()

In [2]:
employee_data = [
    [1,'TONY',101],
    [2,'STEVE',101],
    [3,'BRUCE',102],
    [4,'WANDA',102],
    [5,'VICTOR',103],
    [6,'HANK',105],
]

employee_schema = ["ID", "NAME", "DEPT_ID"]

df_emp =session.createDataFrame(employee_data, schema=employee_schema)
df_emp.show()

-----------------------------
|"ID"  |"NAME"  |"DEPT_ID"  |
-----------------------------
|1     |TONY    |101        |
|2     |STEVE   |101        |
|3     |BRUCE   |102        |
|4     |WANDA   |102        |
|5     |VICTOR  |103        |
|6     |HANK    |105        |
-----------------------------



In [3]:
department_data = [
    [101,'HR'],
    [102,'SALES'],
    [103,'IT'],
    [104,'FINANCE'],
]

department_schema = ["DEPT_ID", "NAME"]

df_dept =session.createDataFrame(department_data, schema=department_schema)
df_dept.show()

-----------------------
|"DEPT_ID"  |"NAME"   |
-----------------------
|101        |HR       |
|102        |SALES    |
|103        |IT       |
|104        |FINANCE  |
-----------------------



In [4]:
from snowflake.snowpark.functions import col

In [5]:
df_emp.filter(col("ID")==1).show()

-----------------------------
|"ID"  |"NAME"  |"DEPT_ID"  |
-----------------------------
|1     |TONY    |101        |
-----------------------------



In [6]:
df_emp.show()

-----------------------------
|"ID"  |"NAME"  |"DEPT_ID"  |
-----------------------------
|1     |TONY    |101        |
|2     |STEVE   |101        |
|3     |BRUCE   |102        |
|4     |WANDA   |102        |
|5     |VICTOR  |103        |
|6     |HANK    |105        |
-----------------------------



In [7]:

df_emp.filter(col("ID").in_(1,2,3)).show()

-----------------------------
|"ID"  |"NAME"  |"DEPT_ID"  |
-----------------------------
|1     |TONY    |101        |
|2     |STEVE   |101        |
|3     |BRUCE   |102        |
-----------------------------



In [8]:
df_emp.filter(df_emp.col("ID").in_(1,2,3)).show()

-----------------------------
|"ID"  |"NAME"  |"DEPT_ID"  |
-----------------------------
|1     |TONY    |101        |
|2     |STEVE   |101        |
|3     |BRUCE   |102        |
-----------------------------



In [9]:
df_dept_SALES = df_dept.filter(col("NAME")=="SALES").select("DEPT_ID")
df_dept_SALES.show()

-------------
|"DEPT_ID"  |
-------------
|102        |
-------------



In [10]:
df_emp_SALES = df_emp.filter(col("DEPT_ID").in_(df_dept_SALES))
df_emp_SALES.show()

-----------------------------
|"ID"  |"NAME"  |"DEPT_ID"  |
-----------------------------
|3     |BRUCE   |102        |
|4     |WANDA   |102        |
-----------------------------



In [11]:
df_emp.select(col("ID"), col("ID").in_(1,2,3).alias("IS_EXISTS")).show()

----------------------
|"ID"  |"IS_EXISTS"  |
----------------------
|1     |True         |
|2     |True         |
|3     |True         |
|4     |False        |
|5     |False        |
|6     |False        |
----------------------



In [12]:
df1 = df_emp.select(col("ID"), col("ID").in_(1,2,3).alias("IS_EXISTS"))
df1.show()

----------------------
|"ID"  |"IS_EXISTS"  |
----------------------
|1     |True         |
|2     |True         |
|3     |True         |
|4     |False        |
|5     |False        |
|6     |False        |
----------------------



In [13]:
df2 = df1.filter(col("IS_EXISTS")=='False').select("ID")
df2.show()

--------
|"ID"  |
--------
|4     |
|5     |
|6     |
--------



In [14]:
df3 = df_emp.filter(col("ID").in_(df2))
df3.show()

-----------------------------
|"ID"  |"NAME"  |"DEPT_ID"  |
-----------------------------
|6     |HANK    |105        |
|5     |VICTOR  |103        |
|4     |WANDA   |102        |
-----------------------------

